In [19]:
import os
import joblib
import tempfile
import glob
import pickle
import tqdm

import pandas as pd
import numpy as np
import cupy as cp

import matplotlib.pyplot as plt

from sklearn.inspection import permutation_importance
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder, MinMaxScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.decomposition import PCA
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.linear_model import LinearRegression, LogisticRegression, RidgeClassifier
from sklearn.ensemble	import RandomForestClassifier, StackingClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.naive_bayes import GaussianNB, ComplementNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis, LinearDiscriminantAnalysis
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble	import StackingClassifier
from sklearn.neural_network import MLPClassifier
from tabpfn import TabPFNClassifier

import xgboost as xgb
import lightgbm as lgb
import catboost as cat

from tabpfn import TabPFNClassifier

from collections import Counter

import optuna
from optuna.samplers import TPESampler
from optuna.storages import RDBStorage
import optuna.visualization as vis
from sklearn.model_selection import cross_val_score
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTEENN
from imblearn import pipeline

from collections import Counter

from sklearn.metrics import (
	accuracy_score, precision_score, recall_score, f1_score,
	matthews_corrcoef, roc_auc_score, confusion_matrix, fbeta_score,
	classification_report,
)

## Part 1: Re-create Value of the Paper

### Part 1.1 Trying to re-create value of the paper that I believe to be faulty

The dataset has 753 duplicates rows, in the paper, they didn't mention about handling the duplicated values, and in the introduction, they delibertly mention that they collect data from **1025** patients, which is larger than **920**, the actual number of rows if they actually merge the four datasets: **Cleveland**, **Hungarian**, **Switzerland** and **Long Beach V** from UCI dataset.

Please check `process.py` from the github repo I added in the report for more detail of how I created the `heart_raw.csv` dataset.

For this step, rather than trying to create the best models, I try to optimise the parameters to have the accuracy as similar as the paper. Since with the dataset that the paper used, it is very hard not to get `100%` accuracy, especially when they didn't even mentioned handle duplicate data.

From the result below, you can see, that even after trying to optimise the hyperameters, we still have 2/6 models achieve **100%** accuracy.

I able to make `Random Forest`, `Decision Tree` and `XGBoost` to achieve `0.931707`, `0.926829` and `0.926829` accuracy. Which is still higher than the paper but the closest I can get. For `KNN`, the accuracy is still super high compare to the paper. `Logistic Regression` and `Naive Bayes` achieve `100%` accuracy instead.

#### Config

In [204]:
TARGET_SCORES = {
	"Logistic Regression":    {"accuracy":0.8439, "f1":0.8620, "recall":0.9090, "precision":0.8196},
	"Naive Bayes":            {"accuracy":0.8439, "f1":0.8608, "recall":0.9000, "precision":0.8250},
	"K-Nearest Neighbors":    {"accuracy":0.8585, "f1":0.8687, "recall":0.8727, "precision":0.8648},
	"Decision Tree":          {"accuracy":0.9268, "f1":0.9289, "recall":0.8909, "precision":0.9702},
	"Random Forest":          {"accuracy":0.9268, "f1":0.9333, "recall":0.9545, "precision":0.9130},
	"Extreme Gradient Boost": {"accuracy":0.9073, "f1":0.9132, "recall":0.9090, "precision":0.9174}
}
TARGET_CM = {
	"Logistic Regression":    {'tn': 73, 'tp': 100, 'fp': 22, 'fn': 10},
	"Naive Bayes":            {'tn': 74, 'tp': 99,  'fp': 21, 'fn': 11},
	"K-Nearest Neighbors":    {'tn': 80, 'tp': 96,  'fp': 15, 'fn': 14},
	"Decision Tree":          {'tn': 92, 'tp': 98,  'fp': 3,  'fn': 12},
	"Random Forest":          {'tn': 85, 'tp': 105, 'fp': 10, 'fn': 5 },
	"Extreme Gradient Boost": {'tn': 86, 'tp': 100, 'fp': 9,  'fn': 10},
}

In [205]:
MODEL_SPECS = {
  "Decision Tree": {
    "class": DecisionTreeClassifier, "init_args": {"random_state":RANDOM_STATE},
    "search": {
      "criterion":       ("categorical", ["gini", "entropy", "log_loss"]),
      "splitter":        ("categorical", ["best", "random"]),
      "max_depth":       ("int", 1, 200),
      "min_samples_split":("int", 2, 500),
      "min_samples_leaf": ("int", 1, 150),
      "max_features":    ("float", 0.01, 1.0),
      "ccp_alpha":       ("float", 0.0, 0.05)
    }
  },
  "Logistic Regression": {
    "class": LogisticRegression, "init_args": {"max_iter":1000},
    "search": {
      "C": ("float", 1e-6, 1e3),
      "penalty": ("categorical", ["l1", "l2"]),
      "solver": ("categorical", ["liblinear", "saga"])
    }
  },
  "Naive Bayes": {
    "class": GaussianNB, "init_args": {},
		"search": {
			"var_smoothing": ("float", 1e-12, 5e-8),
		}
  },
  "K-Nearest Neighbors": {
    "class": KNeighborsClassifier, "init_args": {},
    "search": {
      "n_neighbors": ("int", 1, 50),
      "weights": ("categorical", ["uniform", "distance"]),
      "metric": ("categorical", ["euclidean", "manhattan", "minkowski"])
    }
  },
  "Random Forest": {
    "class": RandomForestClassifier, "init_args": {"random_state":RANDOM_STATE},
    "search": {
      "n_estimators":    ("int", 2, 1000),
      "max_depth":       ("int", 1, 200),
      "min_samples_split":("int", 2, 100),
      "min_samples_leaf": ("int", 1, 150),
      "max_features":    ("float", 0.01, 1.0)
    }
  },
  "Extreme Gradient Boost": {
    "class": xgb.XGBClassifier, "init_args":{
      "use_label_encoder":False, "eval_metric":"logloss", "random_state":RANDOM_STATE
    },
    "search": {
      "n_estimators":    ("int", 2, 2000),
      "max_depth":       ("int", 1, 20),
      "learning_rate":   ("float", 0.001, 1.0),
      "subsample":       ("float", 0.5, 1.0),
      "colsample_bytree":("float", 0.5, 1.0)
    }
  }
}


#### Handling Data

In [265]:
heart_data = pd.read_csv("data/heart.csv")

categorical_cols = ['sex', 'cp', 'fbs', 'restecg', 'exang', 'slope', 'thal']
numerical_cols = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak', 'ca', 'target']
preprocessor = ColumnTransformer(
	transformers=[
		('num', StandardScaler(), numerical_cols),
		('cat', OneHotEncoder(drop='first'), categorical_cols)
	]
)

heart_data_processed = preprocessor.fit_transform(heart_data)
heart_labels = heart_data['target'].values

In [266]:
X_train, X_val, y_train, y_val = train_test_split(
	heart_data_processed, heart_labels, test_size=0.2, random_state=1883669736
)

#### Train Models

Please check tmp.py in given github repo in the report for the full code, the code below is just to fill in the jupyternotebook

In [ ]:
results = {}
OUTPUT_PKL = "tmp_best_params.pkl"

for name, spec in tqdm.tqdm(MODEL_SPECS.items(), desc="Training"):
	def objective(trial):
		params = {}
		for p, info in spec["search"].items():
			t = info[0]
			if t == "int":
				params[p] = trial.suggest_int(p, info[1], info[2])
			elif t == "float":
				params[p] = trial.suggest_float(p, info[1], info[2])
			else:
				params[p] = trial.suggest_categorical(p, info[1])

		model = spec["class"](**{**spec["init_args"], **params})
		model.fit(X_train, y_train)
		y_pred = model.predict(X_val)

		scores = {
			'accuracy':	 accuracy_score(y_val, y_pred),
			'f1':				 f1_score(y_val, y_pred),
			'recall':		 recall_score(y_val, y_pred),
			'precision': precision_score(y_val, y_pred),
		}

		tn, fp, fn, tp = confusion_matrix(y_val, y_pred).ravel()
		cm = {'tp': tp, 'tn': tn, 'fp': fp, 'fn': fn}

		penalty_scores = sum(
			abs(scores[k] - TARGET_SCORES[name][k])
			for k in TARGET_SCORES[name]
		)
		penalty_cm = sum(
			abs(cm[k] - TARGET_CM[name][k])
			for k in TARGET_CM[name]
		)
		total_penalty = penalty_scores + penalty_cm

		return total_penalty

	study = optuna.create_study(
		direction='minimize',
		storage='sqlite:///tmp.db',
		load_if_exists=True,
		study_name=f"{name}_study",
	)
	study.optimize(
		objective,
		n_trials=15000,
		show_progress_bar=True,
		n_jobs=12,
	)

	best_acc = study.best_value
	best_params = study.best_params
	print(f'Best validation accuracy: {best_acc:.3f}')
	print('Best hyperparameters:', best_params)

	model = spec["class"](**{**spec["init_args"]}, **best_params)
	model.fit(X_train, y_train)

	pred = model.predict(X_val)
	print(classification_report(y_val, pred))

	results[name] = {
		"best_params": best_params,
		"metrics": {
			"accuracy": accuracy_score(y_val, pred),
			"precision": precision_score(y_val, pred),
			"recall": recall_score(y_val, pred),
			"f1": f1_score(y_val, pred),
			"confusion_matrix": confusion_matrix(y_val, pred).ravel().tolist()
		}
	}

if os.path.exists(OUTPUT_PKL):
	with open(OUTPUT_PKL, "rb") as f:
		existing = pickle.load(f)
else:
	existing = {}

for m, r in results.items():
	existing[m] = r["best_params"]

with open(OUTPUT_PKL, "wb") as f:
	pickle.dump(existing, f)
	# pickle.dump({m: r["best_params"] for m, r in results.items()}, f)

for name, res in results.items():
	print(f"\n{name}")
	print("Best Params:", res["best_params"])
	print("Achieved Metrics:", res["metrics"])


#### Train Ensemble

In [268]:
with open("tmp_best_params.pkl", "rb") as f:
	best_params_dict = pickle.load(f)

In [269]:
best_params_dict

{'Decision Tree': {'criterion': 'entropy',
  'splitter': 'best',
  'max_depth': 14,
  'min_samples_split': 72,
  'min_samples_leaf': 3,
  'max_features': 0.3171582474435356,
  'ccp_alpha': 2.549196483465216e-05},
 'Logistic Regression': {'C': 730.4282727393,
  'penalty': 'l2',
  'solver': 'saga'},
 'Naive Bayes': {},
 'K-Nearest Neighbors': {'n_neighbors': 50,
  'weights': 'uniform',
  'metric': 'manhattan'},
 'Random Forest': {'n_estimators': 252,
  'max_depth': 155,
  'min_samples_split': 91,
  'min_samples_leaf': 124,
  'max_features': 0.20835925216178564},
 'Extreme Gradient Boost': {'n_estimators': 38,
  'max_depth': 18,
  'learning_rate': 0.0011512563882411127,
  'subsample': 0.9440971661834675,
  'colsample_bytree': 0.5960531503071775}}

In [270]:
base_estimators = [
	('Naive Bayes', GaussianNB(**best_params_dict["Naive Bayes"])),
	('Logistic Regression', LogisticRegression(**best_params_dict["Logistic Regression"], max_iter=1000)),
	('K-Nearest Neighbors', KNeighborsClassifier(**best_params_dict["K-Nearest Neighbors"])),
	('Decision Tree', DecisionTreeClassifier(random_state=1883669736, **best_params_dict["Decision Tree"])),
	('Random Forest', RandomForestClassifier(random_state=1883669736, **best_params_dict["Random Forest"])),
	('Extreme Gradient Boost', xgb.XGBClassifier(use_label_encoder=False, eval_metric="logloss", random_state=1883669736, **best_params_dict["Extreme Gradient Boost"]))
]

In [271]:
results = {}

In [272]:
for estimator in base_estimators:
	estimator[1].fit(X_train, y_train)
	pred = estimator[1].predict(X_val)

	results[estimator[0]] = {
		"accuracy": accuracy_score(y_val, pred),
		"precision": precision_score(y_val, pred),
		"recall": recall_score(y_val, pred),
		"f1": f1_score(y_val, pred),
		"confusion_matrix": confusion_matrix(y_val, pred).ravel().tolist()
	}


/home/newbee/prjs/deakin/sit720/heart_disease_classification/.venv/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [05:00:50] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [273]:
pd.DataFrame(results)

,Naive Bayes,Logistic Regression,K-Nearest Neighbors,Decision Tree,Random Forest,Extreme Gradient Boost
accuracy,1.0,1.0,0.960976,0.931707,0.926829,0.926829
precision,1.0,1.0,0.944444,0.96875,0.908257,0.87395
recall,1.0,1.0,0.980769,0.894231,0.951923,1.0
f1,1.0,1.0,0.962264,0.93,0.929577,0.932735
confusion_matrix,"[101, 0, 0, 104]","[101, 0, 0, 104]","[95, 6, 2, 102]","[98, 3, 11, 93]","[91, 10, 5, 99]","[86, 15, 0, 104]"


In [184]:
meta_clf = LogisticRegression(max_iter=1000)

In [186]:
stacking_clf = StackingClassifier(
	estimators=base_estimators,
	final_estimator=meta_clf,
	passthrough=False,
	cv=5
)

In [274]:
stacking_clf.fit(X_train, y_train)

y_pred_stack = stacking_clf.predict(X_val)

results["Stack"] = {
	"accuracy": accuracy_score(y_val, y_pred_stack),
	"f1": f1_score(y_val, y_pred_stack),
	"recall": recall_score(y_val, y_pred_stack),
	"precision": precision_score(y_val, y_pred_stack),
	"confusion_matrix": confusion_matrix(y_val, pred).ravel().tolist(),
}

/home/newbee/prjs/deakin/sit720/heart_disease_classification/.venv/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [05:00:59] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/newbee/prjs/deakin/sit720/heart_disease_classification/.venv/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [05:01:00] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [198]:
stacking_clf

StackingClassifier(cv=5,
                   estimators=[('Naive Bayes', GaussianNB()),
                               ('Logistic Regression',
                                LogisticRegression(C=730.4282727393,
                                                   max_iter=1000,
                                                   solver='saga')),
                               ('K-Nearest Neighbors',
                                KNeighborsClassifier(metric='manhattan',
                                                     n_neighbors=50)),
                               ('Decision Tree',
                                DecisionTreeClassifier(ccp_alpha=2.549196483465216e-05,
                                                       criterion='entropy',
                                                       max_depth=14,
                                                       max_features=0....
                                              interaction_constraints=None,
                                              learning_rate=0.0011512563882411127,
                                              max_bin=None,
                                              max_cat_threshold=None,
                                              max_cat_to_onehot=None,
                                              max_delta_step=None, max_depth=18,
                                              max_leaves=None,
                                              min_child_weight=None,
                                              missing=nan,
                                              monotone_constraints=None,
                                              multi_strategy=None,
                                              n_estimators=38, n_jobs=None,
                                              num_parallel_tree=None, ...))],
                   final_estimator=LogisticRegression(max_iter=1000))

In [275]:
pd.DataFrame(results)

,Naive Bayes,Logistic Regression,K-Nearest Neighbors,Decision Tree,Random Forest,Extreme Gradient Boost,Stack
accuracy,1.0,1.0,0.960976,0.931707,0.926829,0.926829,1.0
precision,1.0,1.0,0.944444,0.96875,0.908257,0.87395,1.0
recall,1.0,1.0,0.980769,0.894231,0.951923,1.0,1.0
f1,1.0,1.0,0.962264,0.93,0.929577,0.932735,1.0
confusion_matrix,"[101, 0, 0, 104]","[101, 0, 0, 104]","[95, 6, 2, 102]","[98, 3, 11, 93]","[91, 10, 5, 99]","[86, 15, 0, 104]","[86, 15, 0, 104]"


### Part 1.2: Try to train a normal model based on the paper's architecture from the Kaggle's Datset

To prove my point further, using the same dataset the paper was using, without modifying anything, train the models using their default parametrs, all models has `100%` accuracy except for Logistic Regression with `99%` accuracy.

In [276]:
RANDOM_STATE = 384

In [278]:
base_estimators_2 = [
	('2 Naive Bayes', GaussianNB()),
	('2 Logistic Regression', LogisticRegression(max_iter=1000)),
	('2 K-Nearest Neighbors', KNeighborsClassifier()),
	('2 Decision Tree', DecisionTreeClassifier(random_state=RANDOM_STATE)),
	('2 Random Forest', RandomForestClassifier(random_state=RANDOM_STATE)),
	('2 Extreme Gradient Boost', xgb.XGBClassifier(use_label_encoder=False, eval_metric="logloss", random_state=RANDOM_STATE)),
]

In [279]:
for estimator in base_estimators_2:
	estimator[1].fit(X_train, y_train)
	pred = estimator[1].predict(X_val)

	results[estimator[0]] = {
		"accuracy": accuracy_score(y_val, pred),
		"precision": precision_score(y_val, pred),
		"recall": recall_score(y_val, pred),
		"f1": f1_score(y_val, pred),
		"confusion_matrix": confusion_matrix(y_val, pred).ravel().tolist()
	}

/home/newbee/prjs/deakin/sit720/heart_disease_classification/.venv/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [05:01:36] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [280]:
meta_clf_2 = LogisticRegression(max_iter=1000)

stacking_clf_2 = StackingClassifier(
	estimators=base_estimators_2,
	final_estimator=meta_clf_2,
	passthrough=False,
	cv=5
)

In [281]:
stacking_clf.fit(X_train, y_train)

y_pred_stack_2 = stacking_clf.predict(X_val)

results["Stack_2"] = {
	"accuracy": accuracy_score(y_val, y_pred_stack),
	"f1": f1_score(y_val, y_pred_stack),
	"recall": recall_score(y_val, y_pred_stack),
	"precision": precision_score(y_val, y_pred_stack),
	"confusion_matrix": confusion_matrix(y_val, pred).ravel().tolist(),
}

/home/newbee/prjs/deakin/sit720/heart_disease_classification/.venv/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [05:02:38] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/newbee/prjs/deakin/sit720/heart_disease_classification/.venv/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [05:02:39] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [282]:
pd.DataFrame(results)

,Naive Bayes,Logistic Regression,K-Nearest Neighbors,Decision Tree,Random Forest,Extreme Gradient Boost,Stack,2 Naive Bayes,2 Logistic Regression,2 K-Nearest Neighbors,2 Decision Tree,2 Random Forest,2 Extreme Gradient Boost,Stack_2
accuracy,1.0,1.0,0.960976,0.931707,0.926829,0.926829,1.0,1.0,1.0,0.995122,1.0,1.0,1.0,1.0
precision,1.0,1.0,0.944444,0.96875,0.908257,0.87395,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
recall,1.0,1.0,0.980769,0.894231,0.951923,1.0,1.0,1.0,1.0,0.990385,1.0,1.0,1.0,1.0
f1,1.0,1.0,0.962264,0.93,0.929577,0.932735,1.0,1.0,1.0,0.995169,1.0,1.0,1.0,1.0
confusion_matrix,"[101, 0, 0, 104]","[101, 0, 0, 104]","[95, 6, 2, 102]","[98, 3, 11, 93]","[91, 10, 5, 99]","[86, 15, 0, 104]","[86, 15, 0, 104]","[101, 0, 0, 104]","[101, 0, 0, 104]","[101, 0, 1, 103]","[101, 0, 0, 104]","[101, 0, 0, 104]","[101, 0, 0, 104]","[101, 0, 0, 104]"


### Part 1.3: Trying to Re-create paper's models from the actual dataset that they have used

The model here was train by the same file `train.py`, that will be on the github repo mentioned in the report, similar to **Part 2.1**. The only different that `og` will be set to `True` for function `optimise_pipeline`

#### Config

In [217]:
RANDOM_STATE = 384
DATA_PATH = "data/heart_raw.csv"
MODELS_DIR = "models_og"
METRICS_DIR = "metrics_og"
PLOTS_DIR = "plots_og"
STUDY_DB = "studies_og.sqlite"
MIN_ENSEMBLE_SIZE = 4
TOP_K_FEATURES = 10	

#### Utils Functions

In [10]:
mapping = {
	# -- Transformer --
	'tabfn': (
		TabPFNClassifier,
		{
			'n_estimators': (1, 20),
			'softmax_temperature': (0.1, 2.0),
			'balance_probabilities': [False, True],
			'average_before_softmax': [False, True],
		}
	),
	# -- Linear --
	'logistic_regression': (
		LogisticRegression,
		{
			'C': (0.01, 1000.0),
			'penalty': ['l1', 'l2'],
			'solver': ['liblinear', 'saga'],
			'class_weight': [None, 'balanced'],
			'max_iter': [10000],
		}
	),
	'ridge': (
		RidgeClassifier,
		{
			'alpha': (0.001, 100.0),
			'solver': ['auto', 'svd', 'cholesky'],
			'tol': (1e-4, 1e-1),
			'max_iter': [10000],
		}
	),
	'linear_svm': (
		LinearSVC,
		{
			'C': (0.01, 1000.0),
			'loss': ['hinge', 'squared_hinge'],
			'tol': (1e-4, 1e-1),
			'max_iter': [10000],
		}
	),
	# -- Tree --
	'decision_tree': (
		DecisionTreeClassifier,
		{
			'max_depth': (2, 50),
			'min_samples_split': (2, 32),
			'min_samples_leaf': (1, 16),
			'criterion': ['gini', 'entropy'],
			'class_weight': [None, 'balanced']
		}
	),
	'random_forest': (
		RandomForestClassifier,
		{
			'n_estimators': (50, 2500),
			'max_depth': (2, 50),
			'min_samples_split': (2, 32),
			'min_samples_leaf': (1, 16),
			'max_features': ['sqrt', 'log2', 0.5, 0.25, 0.75],
			'criterion': ['gini', 'entropy'],
			'class_weight': [None, 'balanced']
		}
	),
	'xgboost': (
		xgb.XGBClassifier,
		{
			'n_estimators': (50, 2500),
			'max_depth': (2, 50),
			'learning_rate': (0.001, 0.3),
			'subsample': (0.5, 1.0),
			'colsample_bytree': (0.5, 1.0),
			'gamma': (0.0, 5.0),
			'reg_alpha': (0.0, 1.0),
			'reg_lambda': (0.0, 1.0),
		}
	),
	'lightgbm': (
		lgb.LGBMClassifier,
		{
			'num_leaves': (31, 512),
			'learning_rate': (0.01, 0.3),
			'n_estimators': (50, 2000),
			'subsample': (0.5, 1.0),
			'colsample_bytree': (0.5, 1.0),
			'reg_alpha': (0.0, 1.0),
			'reg_lambda': (0.0, 1.0),
			'verbose': [-2],
		}
	),
	'catboost': (
		cat.CatBoostClassifier,
		{
			'iterations': (100, 2000),
			'learning_rate': (0.01, 0.3),
			'depth': (2, 16),
			'l2_leaf_reg': (1, 30),
			'bagging_temperature': (0.0, 1.0),
			'border_count':  (32, 256),
			'verbose': [0],
		}
	),
	'adaboost': (
		AdaBoostClassifier,
		{
			'n_estimators':   (50, 1000),
			'learning_rate':  (0.01, 1.5),
		}
	),
	# -- Generative --
	'gaussian_nb': (
		GaussianNB,
		{
			'var_smoothing': (1e-12, 5e-8)
		}
	),
	'qda': (
		QuadraticDiscriminantAnalysis,
		{
			'reg_param': (0.0, 1.0),
			'tol': (1e-6, 1e-1),
		}
	),
	'lda': (
		LinearDiscriminantAnalysis,
		{
			'solver': ['svd', 'lsqr'],
			'tol': (1e-6, 1e-1),
		}
	),
	# -- Kernel/Neural: non-linear decision boundaries  --
	'knn': (
		KNeighborsClassifier,
		{
			'n_neighbors': (2, 32),
			'weights': ['uniform', 'distance'],
			'metric': ['euclidean', 'manhattan']
		}
	),
	'mlp': (
		MLPClassifier,
		{
			'hidden_layer_sizes':   [(64,), (128,), (64, 64), (128, 64), (64, 128)],
			'activation':           ['relu', 'tanh'],
			'solver':               ['adam', 'sgd'],
			'alpha':                (1e-5, 1e-2),
			'learning_rate_init':   (1e-4, 1e-1),
			'max_iter':             [1000],
		}
	),
	'svm': (
		SVC,
		{
			'C':            (0.01, 1000.0),
			'kernel':       ['linear', 'rbf', 'poly'],
			'gamma':        ['scale', 'auto'],
			'degree':       (2, 5),
			'probability':  [True],
		}
	),
}

In [11]:
class ToGpuTransformer(BaseEstimator, TransformerMixin):
	def __init__(self):
		self._y = None

	def fit(self, X, y=None):
		self._y = y
		return self

	def transform(self, X):
		return cp.array(X) 

def optimise_pipeline(
	model_cls,
	model_param_grid,
	X,
	y,
	study_name,
	sqlite_file,
	n_trials,
	random_state=RANDOM_STATE,
	n_jobs=2,
	og=False,
):
	storage_url = f'sqlite:///{sqlite_file}'
	storage = RDBStorage(url=storage_url)
	try:
		optuna.delete_study(study_name=study_name, storage=storage)
	except KeyError:
		pass

	optuna_sampler = TPESampler()
	study = optuna.create_study(
		# directions=['maximize', 'maximize', 'maximize'],
		direction='maximize',
		sampler=optuna_sampler,
		storage=storage_url,
		study_name=study_name,
		load_if_exists=False,
	)
	cache_dir = tempfile.mkdtemp()

	def objective(trial):
		# -- scaler --
		scaler = 'passthrough'
		scaler_name = trial.suggest_categorical('scaler',
			['standard', 'minmax']
		)
		if scaler_name == 'standard':
			scaler = StandardScaler()
		elif scaler_name == 'minmax':
			scaler = MinMaxScaler()

		if not og:
			# -- variance threshold --
			vt = trial.suggest_float('var_thresh', 0.0, 0.1)
			var_sel = VarianceThreshold(threshold=vt)

			# -- PCA --
			pca = 'passthrough'
			if trial.suggest_categorical('use_pca', [False, True]):
				ratio = trial.suggest_float('pca_ratio', 0.5, 1.0)
				pca = PCA(n_components=ratio, random_state=random_state)

			# -- sampler --
			samp = trial.suggest_categorical('sampler',
				['none', 'smote', 'undersample', 'smoteenn']
			)

			if samp == 'smote':
				sampler = SMOTE(
					random_state=random_state,
					k_neighbors=trial.suggest_int('smote_k', 1, 10),
				)
			elif samp == 'undersample':
				sampler = RandomUnderSampler(
					random_state=random_state,
				)
			elif samp == 'smoteenn':
				sm = SMOTE(
					random_state=random_state,
					k_neighbors=trial.suggest_int('smoteenn_k', 1, 10),
				)
				sampler = SMOTEENN(random_state=random_state, smote=sm)
			else:
				sampler = 'passthrough'
		else:
			var_sel = 'passthrough'
			pca = 'passthrough'
			sampler = 'passthrough'

		# -- model params --
		model_params = {}
		for name, spec in model_param_grid.items():
			if isinstance(spec, list):
				model_params[name] = trial.suggest_categorical(name, spec)
			elif isinstance(spec, tuple) and len(spec) == 2:
				low, high = spec
				if isinstance(low, int) and isinstance(high, int):
					model_params[name] = trial.suggest_int(name, low, high)
				else:
					model_params[name] = trial.suggest_float(name, low, high)
			else:
				raise ValueError(f'Bad spec for {name}: {spec}')

		if model_cls is xgb.XGBClassifier:
			model_params.setdefault('tree_method', 'hist')
			model_params.setdefault('device', 'cuda')
			to_gpu = ToGpuTransformer()
			# to_gpu = 'passthrough'
		elif model_cls is lgb.LGBMClassifier:
			model_params.setdefault('device', 'gpu')
			model_params.setdefault('gpu_platform_id', 0)
			model_params.setdefault('gpu_device_id', 0)
			to_gpu = 'passthrough'
		else:
			to_gpu = 'passthrough'

		try:
			default_params = model_cls().get_params()
			if 'n_jobs' in default_params:
				model.set_params(n_jobs=n_jobs)
		except Exception:
			pass
		model = model_cls(**model_params)

		pipe = pipeline.Pipeline([
			('scaler', scaler),
			('variance', var_sel),
			('pca', pca),
			('sampler', sampler),
			('to_gpu', to_gpu),
			('model', model),
		], memory=cache_dir)

		cv = StratifiedKFold(
			n_splits=5, shuffle=True,
			random_state=random_state
		)

		# AuC to measure model's ability rank true cases above non-cases across all thresholds
		# Similar to F1 but emphasises on Recall, since for heart disease detection, false negatives are costlier than false positives
		# MCC accounts for all four confusion-matrix cells and remains robust on imbalanced datasets
		roc_auc_scores = []
		fbeta_scores = []
		mcc_scores = []
		for tr_idx, va_idx in cv.split(X, y):
			pipe.fit(X.iloc[tr_idx], y.iloc[tr_idx])
			pred = pipe.predict(X.iloc[va_idx])

			fbeta_scores.append(fbeta_score(y.iloc[va_idx], pred, beta=2))

		# return float(np.mean(roc_auc_scores)), float(np.mean(fbeta_scores)), float(np.mean(mcc_scores))
		return float(np.mean(fbeta_scores))

	study.optimize(objective, n_trials=n_trials, show_progress_bar=True, n_jobs=n_jobs)
	return study

In [12]:
def optimise_pipeline(
	model_cls,
	model_param_grid,
	X,
	y,
	study_name,
	sqlite_file,
	n_trials,
	random_state=RANDOM_STATE,
	n_jobs=2,
	og=False,
):
	storage_url = f'sqlite:///{sqlite_file}'
	storage = RDBStorage(url=storage_url)
	try:
		optuna.delete_study(study_name=study_name, storage=storage)
	except KeyError:
		pass

	optuna_sampler = TPESampler()
	study = optuna.create_study(
		# directions=['maximize', 'maximize', 'maximize'],
		direction='maximize',
		sampler=optuna_sampler,
		storage=storage_url,
		study_name=study_name,
		load_if_exists=False,
	)
	cache_dir = tempfile.mkdtemp()

	def objective(trial):
		# -- scaler --
		scaler = 'passthrough'
		scaler_name = trial.suggest_categorical('scaler',
			['standard', 'minmax']
		)
		if scaler_name == 'standard':
			scaler = StandardScaler()
		elif scaler_name == 'minmax':
			scaler = MinMaxScaler()

		if not og:
			# -- variance threshold --
			vt = trial.suggest_float('var_thresh', 0.0, 0.1)
			var_sel = VarianceThreshold(threshold=vt)

			# -- PCA --
			pca = 'passthrough'
			if trial.suggest_categorical('use_pca', [False, True]):
				ratio = trial.suggest_float('pca_ratio', 0.5, 1.0)
				pca = PCA(n_components=ratio, random_state=random_state)

			# -- sampler --
			samp = trial.suggest_categorical('sampler',
				['none', 'smote', 'undersample', 'smoteenn']
			)

			if samp == 'smote':
				sampler = SMOTE(
					random_state=random_state,
					k_neighbors=trial.suggest_int('smote_k', 1, 10),
				)
			elif samp == 'undersample':
				sampler = RandomUnderSampler(
					random_state=random_state,
				)
			elif samp == 'smoteenn':
				sm = SMOTE(
					random_state=random_state,
					k_neighbors=trial.suggest_int('smoteenn_k', 1, 10),
				)
				sampler = SMOTEENN(random_state=random_state, smote=sm)
			else:
				sampler = 'passthrough'
		else:
			var_sel = 'passthrough'
			pca = 'passthrough'
			sampler = 'passthrough'

		# -- model params --
		model_params = {}
		for name, spec in model_param_grid.items():
			if isinstance(spec, list):
				model_params[name] = trial.suggest_categorical(name, spec)
			elif isinstance(spec, tuple) and len(spec) == 2:
				low, high = spec
				if isinstance(low, int) and isinstance(high, int):
					model_params[name] = trial.suggest_int(name, low, high)
				else:
					model_params[name] = trial.suggest_float(name, low, high)
			else:
				raise ValueError(f'Bad spec for {name}: {spec}')

		if model_cls is xgb.XGBClassifier:
			model_params.setdefault('tree_method', 'hist')
			model_params.setdefault('device', 'cuda')
			to_gpu = ToGpuTransformer()
			# to_gpu = 'passthrough'
		elif model_cls is lgb.LGBMClassifier:
			model_params.setdefault('device', 'gpu')
			model_params.setdefault('gpu_platform_id', 0)
			model_params.setdefault('gpu_device_id', 0)
			to_gpu = 'passthrough'
		else:
			to_gpu = 'passthrough'

		try:
			default_params = model_cls().get_params()
			if 'n_jobs' in default_params:
				model.set_params(n_jobs=n_jobs)
		except Exception:
			pass
		model = model_cls(**model_params)

		pipe = pipeline.Pipeline([
			('scaler', scaler),
			('variance', var_sel),
			('pca', pca),
			('sampler', sampler),
			('to_gpu', to_gpu),
			('model', model),
		], memory=cache_dir)

		cv = StratifiedKFold(
			n_splits=5, shuffle=True,
			random_state=random_state
		)

		# AuC to measure model's ability rank true cases above non-cases across all thresholds
		# Similar to F1 but emphasises on Recall, since for heart disease detection, false negatives are costlier than false positives
		# MCC accounts for all four confusion-matrix cells and remains robust on imbalanced datasets
		roc_auc_scores = []
		fbeta_scores = []
		mcc_scores = []
		for tr_idx, va_idx in cv.split(X, y):
			pipe.fit(X.iloc[tr_idx], y.iloc[tr_idx])
			pred = pipe.predict(X.iloc[va_idx])

			fbeta_scores.append(fbeta_score(y.iloc[va_idx], pred, beta=2))

		# return float(np.mean(roc_auc_scores)), float(np.mean(fbeta_scores)), float(np.mean(mcc_scores))
		return float(np.mean(fbeta_scores))

	study.optimize(objective, n_trials=n_trials, show_progress_bar=True, n_jobs=n_jobs)
	return study

In [13]:
def build_pipeline_from_params(model_cls, params, random_state=RANDOM_STATE, n_jobs=2):
	# -- scaler --
	sc = params.get('scaler', 'none')
	if sc == 'standard':
		scaler = StandardScaler()
	elif sc == 'minmax':
		scaler = MinMaxScaler()
	else:
		scaler = 'passthrough'

	# -- variance --
	if 'var_thresh' in params:
		var_sel = VarianceThreshold(threshold=params.get('var_thresh', 0.0))
	else:
		var_sel = 'passthrough'

	# -- PCA --
	if params.get('use_pca', False):
		pca = PCA(n_components=params.get('pca_ratio', 1.0),
							random_state=random_state)
	else:
		pca = 'passthrough'

	# -- sampler --
	samp = params.get('sampler', 'none')
	if samp == 'smote':
		sampler = SMOTE(
			random_state=random_state,
			k_neighbors=params.get('smote_k', 5),
		)
	elif samp == 'undersample':
		sampler = RandomUnderSampler(
			random_state=random_state,
		)
	elif samp == 'smoteenn':
		sm = SMOTE(
			random_state=random_state,
			k_neighbors=params.get('smoteenn_k', 5),
		)
		sampler = SMOTEENN(random_state=random_state, smote=sm)
	else:
		sampler = 'passthrough'

	# model params
	pre_keys = {
		'scaler','var_thresh','use_pca','pca_ratio',
		'sampler','smote_k', 'smoteenn_k',
	}
	model_params = {
		k: v for k, v in params.items() if k not in pre_keys
	}

	if model_cls is xgb.XGBClassifier:
		model_params.setdefault('tree_method', 'hist')
		model_params.setdefault('device', 'cuda')
		to_gpu = ToGpuTransformer()
	else:
		to_gpu = 'passthrough'

	model = model_cls(**model_params)
	try:
		default_params = model_cls().get_params()
		if 'n_jobs' in default_params:
			model.set_params(n_jobs=n_jobs)
	except Exception:
		pass

	return pipeline.Pipeline([
		('scaler', scaler),
		('variance', var_sel),
		('pca', pca),
		('sampler', sampler),
		# ('to_gpu', to_gpu),
		('model', model),
	])


def update_study_map(
	classifier_name, study_name, sqlite_file,
	csv_path='classifier_study_map.csv',
):
	if os.path.exists(csv_path):
		df = pd.read_csv(csv_path)
	else:
		df = pd.DataFrame(columns=['classifier','study','sqlite'])

	entry = {
		'classifier': classifier_name,
		'study': study_name,
		'sqlite': sqlite_file
	}
	df = pd.concat([df, pd.DataFrame([entry])], ignore_index=True)
	df.to_csv(csv_path, index=False)

def evaluate_classification(
	y_true,
	y_pred,
	y_score=None,
	output_csv="metrics.csv"
):
	labels = np.unique(y_true)
	multiclass = labels.size > 2

	# Basic Metrics
	acc = accuracy_score(y_true, y_pred)
	if multiclass:
		f1 = f1_score(y_true, y_pred, average="weighted")
		rec = recall_score(y_true, y_pred, average="weighted")
		prec = precision_score(y_true, y_pred, average="weighted")
	else:
		f1 = f1_score(y_true, y_pred)
		rec = recall_score(y_true, y_pred)
		prec = precision_score(y_true, y_pred)

	# specificity and sensitivity
	if not multiclass:
		tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
		spec = tn / (tn + fp)
		sens = tp / (tp + fn) 
	else:
		cm = confusion_matrix(y_true, y_pred)
		spec_list = []
		for i in range(labels.size):
			tn = cm.sum() - (cm[i, :].sum() + cm[:, i].sum() - cm[i, i])
			fp = cm[:, i].sum() - cm[i, i]
			spec_list.append(tn / (tn + fp))
			sens_list.append(tp / (tp + fn))
		spec = float(np.mean(spec_list))
		sens = float(np.mean(sens_list))

	mcc = matthews_corrcoef(y_true, y_pred)

	# Auc
	if y_score is None:
		auc = np.nan
	else:
		if multiclass:
			auc = roc_auc_score(
				y_true,
				y_score,
				multi_class="ovr",
				average="weighted",
			)
		else:
			# y_score[:, 1] for the positive class
			auc = roc_auc_score(y_true, y_score[:, 1])

	df = pd.DataFrame({
		"Metrics": [
			"Accuracy", "F1-Score", "Recall",
			"Precision", "Sensitivity", "Specificity",
			"MCC", "AUC"
		],
		"Results": [
			acc, f1, rec,
			prec, sens, spec,
			mcc, auc
		]
	})

	df.to_csv(output_csv, index=False)
	return df

In [14]:
def preprocess_data():
	df = pd.read_csv(DATA_PATH)
	X = df.drop(columns=["target"])
	y = (df["target"] > 0).astype(int)

	num_cols = ["age", "trestbps", "chol", "thalach", "oldpeak"]
	cat_cols = ["sex", "cp", "fbs", "restecg", "exang", "slope", "ca", "thal"]

	num_imputer = IterativeImputer(max_iter=100, random_state=RANDOM_STATE)
	cat_imputer = SimpleImputer(strategy="most_frequent")
	# scaler = StandardScaler()
	encoder = OneHotEncoder(sparse_output=False)

	# Impute on full X, then split
	X_num_imp = pd.DataFrame(
		num_imputer.fit_transform(X[num_cols]),
		columns=num_cols
	)
	X_cat_imp = pd.DataFrame(
		cat_imputer.fit_transform(X[cat_cols]),
		columns=cat_cols,
	)
	X_encoded = pd.DataFrame(
		encoder.fit_transform(X_cat_imp),
		columns=encoder.get_feature_names_out(cat_cols)
	)
	X_proc = pd.concat([X_num_imp, X_encoded], axis=1)
	X_train, X_test, y_train, y_test = train_test_split(
		X_proc, y, test_size=0.2,
		random_state=RANDOM_STATE, stratify=y
	)
	return X_train, X_test, y_train, y_test, X_proc.columns.tolist()

In [15]:
def load_models_and_params(features, X_train, X_test, y_test):
  models_dict = {}
  params_dict = {}
  orig_imps = {}
  group_by_ncomp = {}

  for fname in os.listdir(MODELS_DIR):
    if fname.startswith("best_") and fname.endswith("_model.joblib"):
      name = fname.replace("best_", "").replace("_model.joblib", "")
      model_path = os.path.join(MODELS_DIR, fname)
      pipe = joblib.load(model_path)
      models_dict[name] = pipe

      params_path = os.path.join(MODELS_DIR, f"best_{name}_params.joblib")
      best_params = {}
      if os.path.exists(params_path):
        best_params = joblib.load(params_path)
      params_dict[name] = best_params

      model_step = pipe.named_steps.get("model", None)

      if "tabfn" in name:
        tabfn_step = pipe.named_steps["model"]
        if hasattr(tabfn_step, "device"):
          tabfn_step.device = "cuda"

        r = permutation_importance(
          pipe,
          X_test,
          y_test,
          n_repeats=10,
          random_state=RANDOM_STATE,
          n_jobs=-1
        )
        perm_means = r.importances_mean
        feat_imp = {feat: float(perm_means[i]) for i, feat in enumerate(features)}
        orig_imps[name] = feat_imp

        scaler_step = pipe.named_steps.get("scaler", "passthrough")
        if scaler_step != "passthrough":
          X_scaled = scaler_step.transform(X_train)
        else:
          X_scaled = X_train.values

        vt_thresh = best_params.get("var_thresh", 0.0)
        vt = VarianceThreshold(threshold=vt_thresh)
        X_vt = vt.fit_transform(X_scaled)
        n_comp = X_vt.shape[1]
        group_by_ncomp.setdefault(n_comp, []).append(name)
        continue 

      scaler_step = pipe.named_steps.get("scaler", "passthrough")
      if scaler_step != "passthrough":
        X_scaled = scaler_step.transform(X_train)
      else:
        X_scaled = X_train.values

      vt_thresh = best_params.get("var_thresh", 0.0)
      vt = VarianceThreshold(threshold=vt_thresh)
      X_vt = vt.fit_transform(X_scaled)
      vt_mask = vt.get_support()
      vt_features = [f for f, keep in zip(features, vt_mask) if keep]

      pca_step = pipe.named_steps.get("pca", None)
      model_step = pipe.named_steps.get("model", None)
      if model_step is None:
        continue

      if isinstance(pca_step, PCA):
        n_comp = pca_step.n_components_
        group_by_ncomp.setdefault(n_comp, []).append(name)

        loadings = np.abs(pca_step.components_)

        if hasattr(model_step, "feature_importances_"):
          comp_imp = model_step.feature_importances_
        elif hasattr(model_step, "coef_"):
          comp_imp = np.abs(model_step.coef_).ravel()
        else:
          continue

        feat_imp = {}
        for idx, feat in enumerate(features):
          if not vt_mask[idx]:
            # dropped by VT
            feat_imp[feat] = 0.0
          else:
            idx_vt = vt_features.index(feat)

            imp_val = float(np.sum(loadings[:, idx_vt] * comp_imp))
            feat_imp[feat] = imp_val
        orig_imps[name] = feat_imp
      else:
        n_comp = X_vt.shape[1]
        group_by_ncomp.setdefault(n_comp, []).append(name)

        if hasattr(model_step, "feature_importances_"):
          vals = model_step.feature_importances_
        elif hasattr(model_step, "coef_"):
          vals = np.abs(model_step.coef_).ravel()
        else:
          r = permutation_importance(
            pipe, X_test, y_test, n_repeats=10,
            random_state=RANDOM_STATE, n_jobs=-1
          )
          imp_means = r.importances_mean
          feat_imp = {feat: float(imp_means[i]) for i, feat in enumerate(features)}
          orig_imps[name] = feat_imp
          continue

        if len(vals) != len(vt_features):
          continue

        # Map back to original features
        feat_imp = {}
        for idx, feat in enumerate(features):
          if not vt_mask[idx]:
            feat_imp[feat] = 0.0
          else:
            idx_vt = vt_features.index(feat)
            feat_imp[feat] = float(vals[idx_vt])
        orig_imps[name] = feat_imp

  return models_dict, params_dict, group_by_ncomp, orig_imps

#### Training

In [218]:
X_train, X_test, y_train, y_test, features = preprocess_data()

In [219]:
models_dict, params_dict, group_by_ncomp, orig_imps = load_models_and_params(features, X_train, X_test, y_test)

In [222]:
base_estimators = [
  ("rf", models_dict["random_forest_leak"]),
  ("xgb", models_dict["xgboost_leak"]),
  ("knn", models_dict["knn_leak"]),
  ("lr", models_dict["logistic_regression_leak"]),
  ("dt", models_dict["decision_tree_leak"]),
  ("gnb", models_dict["gaussian_nb_leak"]),
]

In [240]:
results = {}

for estimator in base_estimators:
	y_pred = estimator[1].predict(X_test)

	results[estimator[0]] = {
		"accuracy": accuracy_score(y_test, y_pred),
		"f1": f1_score(y_test, y_pred),
		"recall": recall_score(y_test, y_pred),
		"precision": precision_score(y_test, y_pred),
		"confusion_matrix": confusion_matrix(y_test, y_pred).ravel().tolist(),
	}


In [226]:
def objective(trial):
  c_val = trial.suggest_loguniform("meta_lr_c", 1e-4, 1e2)

  meta_lr = LogisticRegression(
    C=c_val,
    solver="liblinear",
    max_iter=1000,
    n_jobs=-1
  )

  stack_clf = StackingClassifier(
    estimators=base_estimators,
    final_estimator=meta_lr,
    cv=5,
    n_jobs=-1,
    passthrough=False
  )

  scores = cross_val_score(
    stack_clf,
    X_train,
    y_train,
    cv=5,
    scoring="f1",
    n_jobs=-1
  )

  return scores.mean()

In [230]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100, show_progress_bar=True, n_jobs=2)

[I 2025-06-01 03:32:57,001] A new study created in memory with name: no-name-14df0ad8-4ca9-43db-b834-6d3a486a9b77


  0%|          | 0/100 [00:00<?, ?it/s]

/tmp/ipykernel_1362749/993844650.py:2: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  c_val = trial.suggest_loguniform("meta_lr_c", 1e-4, 1e2)
/tmp/ipykernel_1362749/993844650.py:2: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  c_val = trial.suggest_loguniform("meta_lr_c", 1e-4, 1e2)
[03:33:09] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inpl

[I 2025-06-01 03:33:33,525] Trial 1 finished with value: 0.831454191837105 and parameters: {'meta_lr_c': 1.469452752198377}. Best is trial 1 with value: 0.831454191837105.
[I 2025-06-01 03:33:33,526] Trial 0 finished with value: 0.8335484256334459 and parameters: {'meta_lr_c': 0.7672152758890183}. Best is trial 0 with value: 0.8335484256334459.


[03:33:46] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

[03:33:46] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

[03:33:46] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to

...

[I 2025-06-01 04:07:08,389] Trial 97 finished with value: 0.8391765860853593 and parameters: {'meta_lr_c': 15.150982545811294}. Best is trial 95 with value: 0.8438345955752707.


/tmp/ipykernel_1362749/993844650.py:2: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  c_val = trial.suggest_loguniform("meta_lr_c", 1e-4, 1e2)
/home/newbee/prjs/deakin/sit720/heart_disease_classification/.venv/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1271: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 16.
  warnings.warn(
/home/newbee/prjs/deakin/sit720/heart_disease_classification/.venv/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1271: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 16.
  warnings.warn(
/home/newbee/prjs/deakin/sit720/heart_disease_classification/.venv/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1271: UserWarning: 'n_jobs' > 1 does not have a

[I 2025-06-01 04:07:44,928] Trial 98 finished with value: 0.8390751565194738 and parameters: {'meta_lr_c': 19.48361052959943}. Best is trial 95 with value: 0.8438345955752707.


/home/newbee/prjs/deakin/sit720/heart_disease_classification/.venv/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1271: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 16.
  warnings.warn(
/home/newbee/prjs/deakin/sit720/heart_disease_classification/.venv/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1271: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 16.
  warnings.warn(
/home/newbee/prjs/deakin/sit720/heart_disease_classification/.venv/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1271: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 16.
  warnings.warn(
/home/newbee/prjs/deakin/sit720/heart_disease_classification/.venv/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1271: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinea

[I 2025-06-01 04:07:47,575] Trial 99 finished with value: 0.8394565994837176 and parameters: {'meta_lr_c': 19.67539493980746}. Best is trial 95 with value: 0.8438345955752707.


In [231]:
best_c = study.best_params["meta_lr_c"]
print(f"Best meta‐learner C: {best_c:.5f}")

Best meta‐learner C: 76.70258


In [232]:
final_meta_lr = LogisticRegression(
  C=best_c,
  solver="liblinear",
  max_iter=1000,
  n_jobs=-1
)

final_stack = StackingClassifier(
  estimators=base_estimators,
  final_estimator=final_meta_lr,
  cv=5,
  n_jobs=-1,
  passthrough=False
)

In [233]:
%%time
final_stack.fit(X_train, y_train)

/home/newbee/prjs/deakin/sit720/heart_disease_classification/.venv/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1271: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 16.
  warnings.warn(


CPU times: user 36.4 ms, sys: 107 ms, total: 143 ms
Wall time: 2.35 s


/home/newbee/prjs/deakin/sit720/heart_disease_classification/.venv/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1271: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 16.
  warnings.warn(


StackingClassifier(cv=5,
                   estimators=[('rf',
                                Pipeline(steps=[('scaler', StandardScaler()),
                                                ('variance', 'passthrough'),
                                                ('pca', 'passthrough'),
                                                ('sampler', 'passthrough'),
                                                ('model',
                                                 RandomForestClassifier(criterion='entropy',
                                                                        max_depth=31,
                                                                        max_features='log2',
                                                                        min_samples_leaf=2,
                                                                        min_samples_split=17,
                                                                        n_estimators=529,
                                                                        n_jobs=12))])),
                               ('xgb',
                                Pipeline(steps=[('scaler'...
                                                                        max_depth=24,
                                                                        min_samples_leaf=15,
                                                                        min_samples_split=20))])),
                               ('gnb',
                                Pipeline(steps=[('scaler', 'passthrough'),
                                                ('variance', 'passthrough'),
                                                ('pca', 'passthrough'),
                                                ('sampler', 'passthrough'),
                                                ('model',
                                                 GaussianNB(var_smoothing=4.9638982978838956e-08))]))],
                   final_estimator=LogisticRegression(C=76.70257832276542,
                                                      max_iter=1000, n_jobs=-1,
                                                      solver='liblinear'),
                   n_jobs=-1)

In [ ]:
y_pred_stack = final_stack.predict(X_test)

results["Stack"] = {
	"accuracy": accuracy_score(y_test, y_pred_stack),
	"f1": f1_score(y_test, y_pred_stack),
	"recall": recall_score(y_test, y_pred_stack),
	"precision": precision_score(y_test, y_pred_stack),
	"confusion_matrix": confusion_matrix(y_test, y_pred_stack).ravel().tolist(),
}

In [245]:
pd.DataFrame(results)

,rf,xgb,knn,lr,dt,gnb,Stack
accuracy,0.858696,0.86413,0.831522,0.826087,0.798913,0.820652,0.869565
f1,0.875,0.880383,0.850242,0.84466,0.817734,0.830769,0.883495
recall,0.892157,0.901961,0.862745,0.852941,0.813725,0.794118,0.892157
precision,0.858491,0.859813,0.838095,0.836538,0.821782,0.870968,0.875
confusion_matrix,"[67, 15, 11, 91]","[67, 15, 10, 92]","[65, 17, 14, 88]","[65, 17, 15, 87]","[64, 18, 19, 83]","[70, 12, 21, 81]","[69, 13, 11, 91]"


## Part 2: Custom Models

#### Config

In [2]:
RANDOM_STATE = 384
DATA_PATH = "data/heart_raw.csv"
MODELS_DIR = "models"
METRICS_DIR = "metrics"
PLOTS_DIR = "plots"
STUDY_DB = "studies.sqlite"
MIN_ENSEMBLE_SIZE = 4
TOP_K_FEATURES = 10

### Part 2.1 Train Models Before Ensembling

#### Training

Please check the link to my github about the trianing of all component of the ensembled models. Since I created a `train.py` instead since the training include various types of models and also take quite a long time for each model, so I was not able to include it in the jupyternotebook.

#### Comparing Metrics

In [135]:
metrics_files = glob.glob("metrics/*_metrics.csv")

In [154]:
data = []

for filepath in metrics_files:
	model_name = os.path.basename(filepath).replace("_metrics.csv", "")
	df = pd.read_csv(filepath)
	row = {"Model": model_name}
	for _, r in df.iterrows():
		row[r["Metrics"]] = r["Results"]
	data.append(row)

pd.DataFrame(data)

,Model,Accuracy,F1-Score,Recall,Precision,Sensitivity,Specificity,MCC,AUC
0,logistic_regression_leak,0.842391,0.859903,0.872549,0.847619,0.872549,0.804878,0.680239,0.917504
1,knn_leak,0.793478,0.820755,0.852941,0.790909,0.852941,0.719512,0.580276,0.880500
2,gaussian_nb_leak,0.869565,0.888889,0.941176,0.842105,0.941176,0.780488,0.738824,0.941416
3,xgboost_leak,0.858696,0.877358,0.911765,0.845455,0.911765,0.792683,0.714074,0.916547
4,random_forest_leak,0.831522,0.855814,0.901961,0.814159,0.901961,0.743902,0.659446,0.904472
5,mlp_leak,0.793478,0.817308,0.833333,0.801887,0.833333,0.743902,0.580577,0.871533
6,adaboost_leak,0.782609,0.809524,0.833333,0.787037,0.833333,0.719512,0.558074,0.877391
7,catboost_leak,0.782609,0.809524,0.833333,0.787037,0.833333,0.719512,0.558074,0.877630
8,lightgbm_leak,0.798913,0.822967,0.843137,0.803738,0.843137,0.743902,0.591476,0.881456
9,svm_leak,0.798913,0.821256,0.833333,0.809524,0.833333,0.756098,0.591877,0.853718


### Part 2.2 Ensembling Models

#### Utils Functions

In [3]:
def select_diverse_models(orig_imps, start_idx=0, overlap_thresh=0.0):
  top_feats = {}
  for name, imps in orig_imps.items():
    sorted_feats = sorted(imps.items(), key=lambda x: x[1], reverse=True)[:TOP_K_FEATURES]
    top_feats[name] = {f: imp for f, imp in sorted_feats}

  sum_scores = {name: sum(top_feats[name].values()) for name in orig_imps}
  ordered = sorted(sum_scores.keys(), key=lambda x: sum_scores[x], reverse=True)
  if not ordered:
    return []

  if start_idx < 0 or start_idx >= len(ordered):
    idx = 0
  else:
    idx = start_idx
  selected = [ordered[idx]]
  sel_feats = set(top_feats[selected[0]])

  candidates = [m for i, m in enumerate(ordered) if i != idx]

  while candidates:
    best_cand = None
    for cand in candidates:
      cand_feats = set(top_feats[cand].keys())
      overlap_feats = sel_feats & cand_feats
      overlap_weight = sum(orig_imps[cand][f] for f in overlap_feats)
      if overlap_weight <= overlap_thresh:
        best_cand = cand
        break

    if best_cand is None:
      break

    selected.append(best_cand)
    sel_feats |= set(top_feats[best_cand].keys())
    candidates.remove(best_cand)

  if len(selected) < MIN_ENSEMBLE_SIZE:
    for name in ordered:
      if name in selected:
        continue
      selected.append(name)
      if len(selected) >= MIN_ENSEMBLE_SIZE:
        break

  return selected

In [4]:
def plot_feature_importances(orig_imps):
  os.makedirs(PLOTS_DIR, exist_ok=True)
  for name, imp_dict in orig_imps.items():
    sorted_feats = sorted(
      imp_dict.items(), key=lambda x: x[1], reverse=True
    )[:TOP_K_FEATURES]
    if not sorted_feats:
      continue

    feats, vals = zip(*sorted_feats)
    plt.figure(figsize=(8, 6))
    plt.barh(range(len(feats))[::-1], vals[::-1])
    plt.yticks(range(len(feats))[::-1], feats[::-1], fontsize=8)
    plt.xlabel("Importance")
    plt.title(f"Top {TOP_K_FEATURES} original features: {name}")
    outp = os.path.join(PLOTS_DIR, f"{name}_features.png")
    plt.tight_layout()
    plt.savefig(outp)
    plt.close()

In [34]:
def plot_combined_feature_importances(orig_imps, top_k=TOP_K_FEATURES // 2):
	os.makedirs(PLOTS_DIR, exist_ok=True)
	df = pd.DataFrame(orig_imps).fillna(0)

	feature_ranks = df.abs().sum(axis=1).sort_values(ascending=False)

	top_features = feature_ranks.index[:top_k]
	df_top = df.loc[top_features]

	plt.figure(figsize=(12, 6))
	df_top.plot.barh(figsize=(12, 6))
	plt.xlabel('Importance')
	plt.ylabel('Feature')
	plt.title(f'Top {top_k} features across all models')
	plt.gca().invert_yaxis()
	plt.legend(title='Model', bbox_to_anchor=(1.05, 1), loc='upper left')
	plt.tight_layout()

	outp = os.path.join(PLOTS_DIR, 'combined_feature_importances.png')
	plt.savefig(outp)
	plt.close()

In [6]:
mapping = {
	# -- Linear --
	'ridge': (
		RidgeClassifier,
		{
			'alpha': (0.1, 100.0),
			'solver': ['auto', 'svd', 'cholesky'],
			'tol': (1e-4, 1e-1),
			'max_iter': [1000],
		}
	),
	# -- Tree --
	'decision_tree': (
		DecisionTreeClassifier,
		{
			'max_depth': (2, 25),
			'min_samples_split': (2, 32),
			'min_samples_leaf': (1, 16),
			'criterion': ['gini', 'entropy'],
			'class_weight': [None, 'balanced']
		}
	),
	'random_forest': (
		RandomForestClassifier,
		{
			'n_estimators': (50, 500),
			'max_depth': (2, 25),
			'min_samples_split': (2, 32),
			'min_samples_leaf': (1, 16),
			'max_features': ['sqrt', 'log2', 0.5, 0.25, 0.75],
			'criterion': ['gini', 'entropy'],
			'class_weight': [None, 'balanced']
		}
	),
}

In [7]:
def optimise_stacking(
	models_dict,
	orig_imps,
	X,
	y,
	study_name,
	sqlite_file,
	n_trials,
	n_jobs
):
	top_feats = {}
	for name, imps in orig_imps.items():
		sorted_feats = sorted(imps.items(), key=lambda x: x[1], reverse=True)[:TOP_K_FEATURES]
		top_feats[name] = {f: imp for f, imp in sorted_feats}

	max_overlap = max(sum(vals.values()) for vals in top_feats.values())

	storage_url = f"sqlite:///{sqlite_file}"
	try:
		optuna.delete_study(study_name=study_name, storage=storage_url)
	except KeyError:
		pass

	sampler = TPESampler()
	study = optuna.create_study(
		direction="maximize",
		sampler=sampler,
		storage=storage_url,
		study_name=study_name,
		load_if_exists=False
	)

	cache_dir = tempfile.mkdtemp()
	candidate_names = list(models_dict.keys())
	num_candidates = len(candidate_names)

	def objective(trial):
		start_idx = trial.suggest_int("start_idx", 0, num_candidates - 1)

		overlap_thresh = trial.suggest_float(
			"overlap_thresh", 0.0, max_overlap
		)

		selected_names = select_diverse_models(
			orig_imps,
			start_idx=start_idx,
			overlap_thresh=overlap_thresh
		)

		final_key = trial.suggest_categorical("final_key", list(mapping.keys()))
		final_cls, param_space = mapping[final_key]

		final_params = {}
		for param_name, spec in param_space.items():
			opt_name = f"{final_key}__{param_name}"
			if isinstance(spec, list):
				final_params[param_name] = trial.suggest_categorical(opt_name, spec)
			elif isinstance(spec, tuple) and len(spec) == 2:
				low, high = spec
				if isinstance(low, int) and isinstance(high, int):
					final_params[param_name] = trial.suggest_int(opt_name, low, high)
				else:
					final_params[param_name] = trial.suggest_float(opt_name, low, high)
			else:
				raise ValueError(f"Bad spec for {param_name}: {spec}")

		if final_key == "xgboost":
			final_params.setdefault("tree_method", "gpu_hist")
			final_params.setdefault("predictor", "gpu_predictor")
			final_params.setdefault("gpu_id", 0)
			final = final_cls(**final_params)
		else:
			final = final_cls(**final_params)

		estimators = [(n, models_dict[n]) for n in selected_names]
		stack = StackingClassifier(
			estimators=estimators,
			final_estimator=final,
			passthrough=False,
			cv=5,
			n_jobs=n_jobs
		)

		cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)
		fbeta_scores = []
		for tr_idx, va_idx in cv.split(X, y):
			X_tr, y_tr = X.iloc[tr_idx], y.iloc[tr_idx]
			X_va, y_va = X.iloc[va_idx], y.iloc[va_idx]

			stack.fit(X_tr, y_tr)
			pred = stack.predict(X_va)

			fbeta_scores.append(fbeta_score(y_va, pred, beta=2))

		return float(np.mean(fbeta_scores))

	study.optimize(
		objective,
		n_trials=n_trials,
		show_progress_bar=True,
		n_jobs=n_jobs
	)
	return study


#### Training Ensembled Models

In this part, I will load all my trained model, calculate feature importants of the model and using Optuna to find the best configuartion of model with diverse feature importants and have the best result.

Optuna will try to optimise final estimators for Stacking Classifier and also optimise way to select diverse model.

I have a function to select diverse model by choosing the "core" model, and arround that core model, I will try to select other models that does not overlap too much of the features with selected models.

In [16]:
X_train, X_test, y_train, y_test, features = preprocess_data()

In [20]:
models_dict, params_dict, group_by_ncomp, orig_imps = load_models_and_params(features, X_train, X_test, y_test)

/home/newbee/prjs/deakin/sit720/heart_disease_classification/.venv/lib/python3.11/site-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


In [35]:
plot_feature_importances(orig_imps)
plot_combined_feature_importances(orig_imps)

<Figure size 1200x600 with 0 Axes>

In [142]:
study = optimise_stacking(
	models_dict=models_dict,
	orig_imps=orig_imps,
	X=X_train,
	y=y_train,
	study_name="stacking_ensemble_study",
	sqlite_file=STUDY_DB,
	n_trials=30,
	n_jobs=8,
)
print("Best Optuna parameters:")
print(study.best_params)

[I 2025-05-31 22:07:52,370] A new study created in RDB with name: stacking_ensemble_study


  0%|          | 0/30 [00:00<?, ?it/s]

/home/newbee/prjs/deakin/sit720/heart_disease_classification/.venv/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1271: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 16.
  warnings.warn(
/home/newbee/prjs/deakin/sit720/heart_disease_classification/.venv/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1271: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 16.
  warnings.warn(
/home/newbee/prjs/deakin/sit720/heart_disease_classification/.venv/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1271: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 16.
  warnings.warn(
/home/newbee/prjs/deakin/sit720/heart_disease_classification/.venv/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1271: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinea

[I 2025-05-31 22:10:51,846] Trial 5 finished with value: 0.8518167892437758 and parameters: {'start_idx': 9, 'overlap_thresh': 46.757046046197175, 'final_key': 'random_forest', 'random_forest__n_estimators': 321, 'random_forest__max_depth': 16, 'random_forest__min_samples_split': 21, 'random_forest__min_samples_leaf': 7, 'random_forest__max_features': 'sqrt', 'random_forest__criterion': 'gini', 'random_forest__class_weight': None}. Best is trial 5 with value: 0.8518167892437758.


/home/newbee/prjs/deakin/sit720/heart_disease_classification/.venv/lib/python3.11/site-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
[22:10:57] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.



...

[I 2025-05-31 22:35:39,148] Trial 26 finished with value: 0.8597841038965568 and parameters: {'start_idx': 0, 'overlap_thresh': 122.93098930842926, 'final_key': 'ridge', 'ridge__alpha': 2.363390740705057, 'ridge__solver': 'auto', 'ridge__tol': 0.0915070281997549, 'ridge__max_iter': 1000}. Best is trial 26 with value: 0.8597841038965568.


/home/newbee/prjs/deakin/sit720/heart_disease_classification/.venv/lib/python3.11/site-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/home/newbee/prjs/deakin/sit720/heart_disease_classification/.venv/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/home/newbee/prjs/deakin/sit720/heart_disease_classification/.venv/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/home/newbee/prjs/deakin/sit720/heart_disease_classification/.venv/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was f

[I 2025-05-31 22:35:51,381] Trial 29 finished with value: 0.8569759670393002 and parameters: {'start_idx': 2, 'overlap_thresh': 118.09631449191778, 'final_key': 'ridge', 'ridge__alpha': 60.03311730160058, 'ridge__solver': 'cholesky', 'ridge__tol': 0.09856300456831192, 'ridge__max_iter': 1000}. Best is trial 26 with value: 0.8597841038965568.


/home/newbee/prjs/deakin/sit720/heart_disease_classification/.venv/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/home/newbee/prjs/deakin/sit720/heart_disease_classification/.venv/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/home/newbee/prjs/deakin/sit720/heart_disease_classification/.venv/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/home/newbee/prjs/deakin/sit720/heart_disease_classification/.venv/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/home/newbee/prj

[I 2025-05-31 22:35:58,260] Trial 28 finished with value: 0.8549562941448239 and parameters: {'start_idx': 0, 'overlap_thresh': 110.80186751494783, 'final_key': 'ridge', 'ridge__alpha': 60.80592592100191, 'ridge__solver': 'cholesky', 'ridge__tol': 0.0982766850561539, 'ridge__max_iter': 1000}. Best is trial 26 with value: 0.8597841038965568.
Best Optuna parameters:
{'start_idx': 0, 'overlap_thresh': 122.93098930842926, 'final_key': 'ridge', 'ridge__alpha': 2.363390740705057, 'ridge__solver': 'auto', 'ridge__tol': 0.0915070281997549, 'ridge__max_iter': 1000}


In [143]:
best_start_idx = study.best_params["start_idx"]
best_overlap = study.best_params["overlap_thresh"]
best_key = study.best_params["final_key"]

selected_models = select_diverse_models(
	orig_imps,
	start_idx=best_start_idx,
	overlap_thresh=best_overlap
)
selected_models

['catboost_leak',
 'lightgbm_leak',
 'logistic_regression_leak',
 'lda_leak',
 'adaboost_leak',
 'ridge_leak',
 'linear_svm_leak',
 'random_forest_leak',
 'xgboost_leak',
 'svm_leak',
 'tabfn_leak',
 'gaussian_nb_leak',
 'qda_leak',
 'mlp_leak',
 'knn_leak']

In [151]:
study.best_params

{'start_idx': 0,
 'overlap_thresh': 122.93098930842926,
 'final_key': 'ridge',
 'ridge__alpha': 2.363390740705057,
 'ridge__solver': 'auto',
 'ridge__tol': 0.0915070281997549,
 'ridge__max_iter': 1000}

In [145]:
final_cls, param_space = mapping[best_key]
final_params = {}
for param_name in param_space:
	opt_name = f"{best_key}__{param_name}"
	final_params[param_name] = study.best_params[opt_name]

final = final_cls(**final_params)

In [146]:
estimators = [(n, models_dict[n]) for n in selected_models]
ensemble = StackingClassifier(
	estimators=estimators,
	final_estimator=final,
	passthrough=False,
	cv=5,
	n_jobs=-1
)
ensemble.fit(X_train, y_train)

/home/newbee/prjs/deakin/sit720/heart_disease_classification/.venv/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1271: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 16.
  warnings.warn(
/home/newbee/prjs/deakin/sit720/heart_disease_classification/.venv/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/home/newbee/prjs/deakin/sit720/heart_disease_classification/.venv/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/home/newbee/prjs/deakin/sit720/heart_disease_classification/.venv/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/h

StackingClassifier(cv=5,
                   estimators=[('catboost_leak',
                                Pipeline(steps=[('scaler', MinMaxScaler()),
                                                ('variance',
                                                 VarianceThreshold(threshold=0.09443732680182998)),
                                                ('pca',
                                                 PCA(n_components=0.5295497970791623,
                                                     random_state=384)),
                                                ('sampler', 'passthrough'),
                                                ('model',
                                                 <catboost.core.CatBoostClassifier object at 0x77f5362b3c90>)])),
                               ('lightgbm_leak',
                                Pipeline(steps=[('...
                                                               max_iter=1000))])),
                               ('knn_leak',
                                Pipeline(steps=[('scaler', MinMaxScaler()),
                                                ('variance',
                                                 VarianceThreshold(threshold=0.048696167293489336)),
                                                ('pca', 'passthrough'),
                                                ('sampler', 'passthrough'),
                                                ('model',
                                                 KNeighborsClassifier(metric='euclidean',
                                                                      n_jobs=16,
                                                                      n_neighbors=31))]))],
                   final_estimator=RidgeClassifier(alpha=2.363390740705057,
                                                   max_iter=1000,
                                                   tol=0.0915070281997549),
                   n_jobs=-1)

In [147]:
from sklearn.utils import estimator_html_repr
html = estimator_html_repr(ensemble)

with open("ensemble_model.html", "w") as f:
	f.write(html)

In [152]:
preds = ensemble.predict(X_test)
try:
	probs = ensemble.predict_proba(X_test)
except Exception:
	probs = None

acc = accuracy_score(y_test, preds)
f1 = f1_score(y_test, preds)
rec = recall_score(y_test, preds)
prec = precision_score(y_test, preds)
tn, fp, fn, tp = confusion_matrix(y_test, preds).ravel()
spec = tn / (tn + fp)
sens = tp / (tp + fn)
mcc = matthews_corrcoef(y_test, preds)
auc = roc_auc_score(y_test, probs[:, 1]) if probs is not None else np.nan

metrics_df = pd.DataFrame({
	"Metrics": [
		"Accuracy", "F1-Score", "Recall",
		"Precision", "Sensitivity", "Specificity",
		"MCC", "AUC"
	],
	"Results": [
		acc, f1, rec, prec, sens, spec, mcc, auc
	]
})

os.makedirs(METRICS_DIR, exist_ok=True)
metrics_path = os.path.join(METRICS_DIR, "stacking_metrics.csv")
metrics_df.to_csv(metrics_path, index=False)
print(f"Stacking metrics saved to {metrics_path}")

metrics_df

/home/newbee/prjs/deakin/sit720/heart_disease_classification/.venv/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


Stacking metrics saved to metrics/stacking_metrics.csv


,Metrics,Results
0,Accuracy,0.858696
1,F1-Score,0.876190
2,Recall,0.901961
3,Precision,0.851852
4,Sensitivity,0.901961
5,Specificity,0.804878
6,MCC,0.713524
7,AUC,NaN


In [153]:
os.makedirs(MODELS_DIR, exist_ok=True)
ensemble_path = os.path.join(MODELS_DIR, "ensemble_model.joblib")
joblib.dump(ensemble, ensemble_path)
print(f"Ensemble model saved to {ensemble_path}")

Ensemble model saved to models/ensemble_model.joblib
